# Lifecycle of Write Query

## Creating a Table

In [1]:
%%sql
CREATE TABLE IF NOT EXISTS catalog.sales.orders (
    order_id BIGINT,
    customer_id BIGINT,
    order_amount DECIMAL(10,2),
    order_ts TIMESTAMP
)
USING iceberg
PARTITIONED BY (day(order_ts))

25/03/25 18:24:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

### Table Metadata
**Metadata file location:** warehouse/default/db/orders/metadata/v1.metadata.json ([File Contents](refs/metadata.json)) It includes the following details:
 - Table identification
 - Schema definition
 - Partitioning
 - Table properties
 - Snapshot tracking
 - Timestamps
    

### Queries to Inspect Iceberg Data Lakehouse

```sql
SHOW NAMESPACES IN catalog;
SHOW TABLES IN catalog.sales;
SELECT * FROM catalog.sales.orders.manifests;
SELECT * FROM catalog.sales.orders.snapshots;
SELECT * FROM catalog.sales.orders.files
SELECT * FROM catalog.sales.orders.partitions
```

If you run any of this queries, you'll find that none of them have any records in it at table creation. 

In [2]:
%%sql
INSERT INTO catalog.sales.orders VALUES (
   100,
   23,
   40,
   CAST('2025-03-17 14:00:00' AS TIMESTAMP)
)

++
||
++
++

In [3]:
%%sql

SELECT * FROM catalog.sales.orders

order_id,customer_id,order_amount,order_ts
100,23,40.00,2025-03-17 14:00:00


In [4]:
%%sql
SELECT * FROM catalog.sales.orders.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-03-25 18:25:28.585000,3472866772545382495,None,append,s3://warehouse/catalog/sales/orders/metadata/snap-3472866772545382495-1-5583a730-0bfb-48ab-91fb-34fd9c0f922c.avro,"{'engine-version': '3.5.5', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1742927076491', 'added-records': '1', 'total-records': '1', 'spark.app.id': 'local-1742927076491', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '1354', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '1354', 'total-data-files': '1'}"


In [5]:
%%sql
SELECT * FROM catalog.sales.orders.manifests

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3://warehouse/catalog/sales/orders/metadata/5583a730-0bfb-48ab-91fb-34fd9c0f922c-m0.avro,7462,0,3472866772545382495,1,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-17')]"


In [6]:
%%sql
SELECT * FROM catalog.sales.orders.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/catalog/sales/orders/data/order_ts_day=2025-03-17/00000-1-95f1c47c-5fd9-4ef2-8a5d-f657feafc707-0-00001.parquet,PARQUET,0,"Row(order_ts_day=datetime.date(2025, 3, 17))",1,1354,"{1: 43, 2: 42, 3: 49, 4: 49}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'd\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x17\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x0f\xa0'), 4: bytearray(b'\x00x84\x8a0\x06\x00')}","{1: bytearray(b'd\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x17\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x0f\xa0'), 4: bytearray(b'\x00x84\x8a0\x06\x00')}",None,[4],None,0,None,None,None,"Row(customer_id=Row(column_size=42, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=23, upper_bound=23), order_amount=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=Decimal('40.00'), upper_bound=Decimal('40.00')), order_id=Row(column_size=43, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=100, upper_bound=100), order_ts=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2025, 3, 17, 14, 0), upper_bound=datetime.datetime(2025, 3, 17, 14, 0)))"


In [7]:
%%sql
SELECT * FROM catalog.sales.orders.partitions

partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
"Row(order_ts_day=datetime.date(2025, 3, 17))",0,1,1,1354,0,0,0,0,2025-03-25 18:25:28.585000,3472866772545382495


In [8]:
%%sql
CREATE OR REPLACE TABLE catalog.sales.orders_staging (
    order_id BIGINT,
    customer_id BIGINT,
    order_amount DECIMAL(10,2),
    order_ts TIMESTAMP    
);

++
||
++
++

In [9]:
%%sql
INSERT INTO catalog.sales.orders_staging
    VALUES (100, 23, 45.2, CAST('2025-03-17 12:30:01' AS TIMESTAMP)),
           (101, 30, 55, CAST('2025-03-18 09:43:37' AS TIMESTAMP))

++
||
++
++

In [10]:
%%sql
SELECT * FROM catalog.sales.orders_staging

order_id,customer_id,order_amount,order_ts
100,23,45.20,2025-03-17 12:30:01
101,30,55.00,2025-03-18 09:43:37


In [11]:
%%sql
MERGE INTO catalog.sales.orders o
USING (SELECT * FROM catalog.sales.orders_staging) s
ON o.order_id = s.order_id
WHEN MATCHED THEN UPDATE SET order_amount = s.order_amount
WHEN NOT MATCHED THEN INSERT *;

++
||
++
++

In [12]:
%%sql
SELECT * FROM catalog.sales.orders

order_id,customer_id,order_amount,order_ts
100,23,45.20,2025-03-17 14:00:00
101,30,55.00,2025-03-18 09:43:37


In [13]:
%%sql
SELECT * FROM catalog.sales.orders.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-03-25 18:25:28.585000,3472866772545382495,None,append,s3://warehouse/catalog/sales/orders/metadata/snap-3472866772545382495-1-5583a730-0bfb-48ab-91fb-34fd9c0f922c.avro,"{'engine-version': '3.5.5', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1742927076491', 'added-records': '1', 'total-records': '1', 'spark.app.id': 'local-1742927076491', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '1354', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '1354', 'total-data-files': '1'}"
2025-03-25 18:26:22.380000,1046485116440928236,3472866772545382495,overwrite,s3://warehouse/catalog/sales/orders/metadata/snap-1046485116440928236-1-6878539c-1bb7-4661-9a7d-5d9739db0080.avro,"{'engine-version': '3.5.5', 'added-data-files': '2', 'total-equality-deletes': '0', 'app-id': 'local-1742927076491', 'added-records': '2', 'deleted-data-files': '1', 'deleted-records': '1', 'total-records': '2', 'spark.app.id': 'local-1742927076491', 'removed-files-size': '1354', 'changed-partition-count': '2', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '2750', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '2750', 'total-data-files': '2'}"


In [14]:
%%sql
SELECT * FROM catalog.sales.orders.manifests

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,s3://warehouse/catalog/sales/orders/metadata/6878539c-1bb7-4661-9a7d-5d9739db0080-m1.avro,7499,0,1046485116440928236,2,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-18')]"
0,s3://warehouse/catalog/sales/orders/metadata/6878539c-1bb7-4661-9a7d-5d9739db0080-m0.avro,7462,0,1046485116440928236,0,0,1,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-17')]"


In [15]:
%%sql
SELECT * FROM catalog.sales.orders.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/catalog/sales/orders/data/order_ts_day=2025-03-17/00000-16-a094f7c5-c4e2-4953-8468-6fdf53452b40-0-00002.parquet,PARQUET,0,"Row(order_ts_day=datetime.date(2025, 3, 17))",1,1375,"{1: 49, 2: 49, 3: 49, 4: 49}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'd\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x17\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x11\xa8'), 4: bytearray(b'\x00x84\x8a0\x06\x00')}","{1: bytearray(b'd\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x17\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x11\xa8'), 4: bytearray(b'\x00x84\x8a0\x06\x00')}",None,[4],None,0,None,None,None,"Row(customer_id=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=23, upper_bound=23), order_amount=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=Decimal('45.20'), upper_bound=Decimal('45.20')), order_id=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=100, upper_bound=100), order_ts=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2025, 3, 17, 14, 0), upper_bound=datetime.datetime(2025, 3, 17, 14, 0)))"
0,s3://warehouse/catalog/sales/orders/data/order_ts_day=2025-03-18/00000-16-a094f7c5-c4e2-4953-8468-6fdf53452b40-0-00001.parquet,PARQUET,0,"Row(order_ts_day=datetime.date(2025, 3, 18))",1,1375,"{1: 49, 2: 49, 3: 49, 4: 49}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'e\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x1e\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x15|'), 4: bytearray(b'@\xe4)\xbd\x9a0\x06\x00')}","{1: bytearray(b'e\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'\x1e\x00\x00\x00\x00\x00\x00\x00'), 3: bytearray(b'\x15|'), 4: bytearray(b'@\xe4)\xbd\x9a0\x06\x00')}",None,[4],None,0,None,None,None,"Row(customer_id=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=30, upper_bound=30), order_amount=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=Decimal('55.00'), upper_bound=Decimal('55.00')), order_id=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=101, upper_bound=101), order_ts=Row(column_size=49, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2025, 3, 18, 9, 43, 37), upper_bound=datetime.datetime(2025, 3, 18, 9, 43, 37)))"


In [30]:
%%sql
SELECT * FROM catalog.sales.orders.all_manifests

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries,reference_snapshot_id
0,s3://warehouse/catalog/sales/orders/metadata/5583a730-0bfb-48ab-91fb-34fd9c0f922c-m0.avro,7462,0,3472866772545382495,1,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-17')]",3472866772545382495
0,s3://warehouse/catalog/sales/orders/metadata/6878539c-1bb7-4661-9a7d-5d9739db0080-m1.avro,7499,0,1046485116440928236,2,0,0,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-18')]",1046485116440928236
0,s3://warehouse/catalog/sales/orders/metadata/6878539c-1bb7-4661-9a7d-5d9739db0080-m0.avro,7462,0,1046485116440928236,0,0,1,0,0,0,"[Row(contains_null=False, contains_nan=False, lower_bound='2025-03-17', upper_bound='2025-03-17')]",1046485116440928236
